In [228]:
import PyPDF2
import re
import pandas as pd
import matplotlib.pyplot as plt

In [239]:
pdf = open('sept18.pdf', 'rb')
reader = PyPDF2.PdfFileReader(pdf)
pages = reader.numPages

In [240]:
# Should be 0, but getting the last page because it's shorter
first_page = reader.getPage(pages - 1)
first_page_text = first_page.extractText()
#info = re.findall(r'\b([\w/.,#\'? - ? ]*)\n', first_page_text)
#info = re.findall(r'([\w/.,#\' ]*)\n', first_page_text)
info = first_page_text.split("\n")

In [241]:
#info

In [242]:
# Remove the beginning data
del info[:5]

# Create a list for the column names
x_axis = info[:7]

# Remove column titles and trailing info
del info[:7]
del info[-4:]

In [243]:
#info

In [244]:
def check_vals(tmp, current_val):
    # Pattern [Group Number, BIN, Program, Date, Utilization, Rate, Ammount]
    if not re.match(r'^[A-Z\d]*$', tmp[0]):
        print("Error: Incorrect Group Number - {}".format(tmp[0]))
        return False
        
    if not re.match(r'^[0-9]*$', tmp[1]):
        print("Error: Incorrect Bin Number - {}".format(tmp[1]))
        return False
    
    if not re.match(r'^[A-Za-z0-9-.\' ]*$', tmp[2]):
        print("Error: Incorrect Program Name - {}".format(tmp[2]))
        return False
    
    if not re.match(r'([0-9]{1,2}\/[0-9]{1,2})', tmp[3]):
        print("Error: Incorrect Date - {}".format(tmp[3]))
        return False
        
    if not re.match(r'^[0-9]*$', tmp[4]):
        print("Error: Incorrect Utilization Number - {}".format(tmp[4]))
        return False
    
    # Clean the $ off the last two columns
    if tmp[5][0] == "$":
        tmp[5] = tmp[5][1:]
        info[current_val - 2] = tmp[5]
        
    if tmp[6][0] == "$":
        tmp[6] = tmp[6][1:]
        info[current_val - 1] = tmp[6]
    
        
    if not re.match(r'^(\d(,\d{3})*|(\d+))(\.\d{2})$', tmp[5]):
        print("Error: Incorrect Rate Number - {}".format(tmp[5]))
        return False
        
    if not re.match(r'^(\d(,\d{3})*|(\d+))(\.\d{2})$', tmp[6]):
        print("Error: Incorrect Payout Number - {}".format(tmp[6]))
        return False
    return True

In [245]:
df = []
for i in range(len(info)):
    if i != 0 and i % 7 == 0:
        # First row
        if i == 7:
            # Check the values of the data
            tmp = info[:7]
            # If the value is okay, append it to the data frame
            if check_vals(tmp, i):
                df.append(info[:7])
            else:
                break
        else:
            # If there is a space after the program name, this is because there is a word wrap.
            # The next index will be the rest of the name, so we need to add the following
            # value to the current one, then delete the following value from the list
            if info[i - 5][-1] == " " or info[i - 5][-1] == "-":
                info[i - 5] = info[i - 5] + info[i - 4]
                del info[i - 4]
            tmp = info[i-7:i]
            if check_vals(tmp, i):
                df.append(info[i-7:i])     
        

In [246]:
comms = pd.DataFrame(df, columns=x_axis)
comms

,Group ID,Bin #,Program,MM/YY,Utilization,Broker Rate,Amount Paid
0,RXNNCV2,610709,NonCoveredDrugs.com,9/18,2,0.60,1.20
1,RXNNCVC,610709,NonCoveredDrugs.com Coupons,9/18,49,0.60,29.40
2,RXNNCVD,610709,Non Covered Drugs,9/18,6,0.60,3.60
3,RXPHRMCRD,610280,RxPharmacyCards.com,9/18,4,0.25,1.00
4,RXPHRMCRD,610709,RxPharmacyCards.com,9/18,974,0.25,243.50
5,RXPNTH,610709,paintherapyrx.com Card,9/18,1,0.60,0.60
6,RXPNTHC,610709,paintherapyrx.com Coupons,9/18,14,0.60,8.40
7,RXSBST,610709,Substance Withdrawal Rx,9/18,0,0.60,0.00
8,RXSKNC,610709,Skin Care Rx Coupons,9/18,0,0.60,0.00
9,RXSLPS,610709,Sleep Savings,9/18,8,0.60,4.80
